In [ ]:
# Install the required Python dependencies.
!pip install -q omegaconf==2.3.0 iopath==0.1.10 timm==0.6.13 webdataset==0.2.48
!pip install -q visual-genome==1.1.1 decord==0.6.0 pysrt==1.1.2 wandb==0.14.2
!pip install -q openai==0.28.0 pycocoevalcap==1.2 pytubefix moviepy==1.0.3
!pip install -q soundfile==0.12.1 sentencepiece==0.1.97 opencv-python==4.7.0.72
!pip install -q scikit-image==0.22.0 webvtt-py==0.4.6 torch==2.0.1 triton==2.0.0
!pip install -q torchaudio==2.0.2 torchvision==0.15.2 transformers==4.37.2 tokenizers==0.15.2
!pip install -q bitsandbytes==0.42.0 peft==0.2.0 accelerate==0.25.0

In [ ]:
!pip install numpy==1.26.3
import numpy as np
print(np.__version__)

In [ ]:
!pip install -q nltk
import nltk
nltk.download('wordnet', download_dir='/root/nltk_data')
nltk.data.path.append('/root/nltk_data')
!huggingface-cli login
# use your own huggingface authentication key and  paste it.

In [ ]:
!git clone https://github.com/camenduru/MiniGPT4-video
%cd MiniGPT4-video
!apt -y install -qq aria2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M \
  https://huggingface.co/Vision-CAIR/MiniGPT4-Video/resolve/main/checkpoints/video_llama_checkpoint_last.pth \
  -d /content/MiniGPT4-video/pretrained_models -o video_llama_checkpoint_last.pth

In [ ]:


# === BACKEND CODE: Model initialization and utility functions ===
import os
import torch
import webvtt
import cv2
from minigpt4.common.eval_utils import init_model
from minigpt4.conversation.conversation import CONV_VISION
from torchvision import transforms
from tqdm import tqdm
import soundfile as sf
import moviepy.editor as mp
from PIL import Image
from moviepy.editor import VideoFileClip
import random
import numpy as np
import torch.backends.cudnn as cudnn
import yaml

class Options:
    def __init__(self):
        self.cfg_path = "/content/MiniGPT4-video/test_configs/llama2_test_config.yaml"
        self.ckpt = '/content/MiniGPT4-video/pretrained_models/video_llama_checkpoint_last.pth'
        self.add_subtitles = False  # Set to True if you want to generate subtitles via whisper.
        self.question = None
        self.video_path = None
        self.max_new_tokens = 512
        self.lora_r = 64
        self.lora_alpha = 16
        self.options = None

args = Options()

def prepare_input(vis_processor, video_path, subtitle_path, instruction):
    cap = cv2.VideoCapture(video_path)
    if subtitle_path is not None:
        vtt_file = webvtt.read(subtitle_path)
        print("Subtitle loaded successfully.")
        clip = VideoFileClip(video_path)
        total_num_frames = int(clip.duration * clip.fps)
        clip.close()
    else:
        total_num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    max_images_length = 45
    max_sub_len = 400
    images = []
    frame_count = 0
    sampling_interval = int(total_num_frames / max_images_length)
    if sampling_interval == 0:
        sampling_interval = 1
    img_placeholder = ""
    subtitle_text_in_interval = ""
    history_subtitles = {}
    raw_frames = []
    number_of_words = 0
    transform = transforms.Compose([transforms.ToPILImage(),])
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if subtitle_path is not None:
            for subtitle in vtt_file:
                sub = subtitle.text.replace('\n',' ')
                if (subtitle.start_in_seconds <= (frame_count / int(clip.fps)) <= subtitle.end_in_seconds) and sub not in subtitle_text_in_interval:
                    if not history_subtitles.get(sub, False):
                        subtitle_text_in_interval += sub + " "
                    history_subtitles[sub] = True
                    break
        if frame_count % sampling_interval == 0:
            raw_frames.append(Image.fromarray(cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2RGB)))
            frame = transform(frame[:,:,::-1])
            frame = vis_processor(frame)
            images.append(frame)
            img_placeholder += '<Img><ImageHere>'
            if subtitle_path is not None and subtitle_text_in_interval != "" and number_of_words < max_sub_len:
                img_placeholder += f'<Cap>{subtitle_text_in_interval}'
                number_of_words += len(subtitle_text_in_interval.split(' '))
                subtitle_text_in_interval = ""
        frame_count += 1
        if len(images) >= max_images_length:
            break
    cap.release()
    cv2.destroyAllWindows()
    if len(images) == 0:
        return None, None
    images = torch.stack(images)
    instruction = img_placeholder + '\n' + instruction
    return images, instruction

def extract_audio(video_path, audio_path):
    video_clip = mp.VideoFileClip(video_path)
    audio_clip = video_clip.audio
    audio_clip.write_audiofile(audio_path, codec="libmp3lame", bitrate="320k")

def generate_subtitles(video_path):
    video_id = video_path.split('/')[-1].split('.')[0]
    audio_path = f"workspace/inference_subtitles/mp3/{video_id}.mp3"
    os.makedirs("workspace/inference_subtitles/mp3", exist_ok=True)
    if existed_subtitles.get(video_id, False):
        return f"workspace/inference_subtitles/{video_id}.vtt"
    try:
        extract_audio(video_path, audio_path)
        print("Audio extracted successfully.")
        os.system(f"whisper {audio_path} --language English --model large --output_format vtt --output_dir workspace/inference_subtitles")
        os.system(f"rm {audio_path}")
        print("Subtitle generated successfully.")
        return f"workspace/inference_subtitles/{video_id}.vtt"
    except Exception as e:
        print("Error:", e)
        print("Error processing", video_path)
        return None

def run(video_path, instruction, model, vis_processor, gen_subtitles=True):
    if gen_subtitles:
        subtitle_path = generate_subtitles(video_path)
    else:
        subtitle_path = None
    prepared_images, prepared_instruction = prepare_input(vis_processor, video_path, subtitle_path, instruction)
    if prepared_images is None:
        return "Video cannot be opened. Please check the video file."
    length = len(prepared_images)
    prepared_images = prepared_images.unsqueeze(0)
    conv = CONV_VISION.copy()
    conv.system = ""
    conv.append_message(conv.roles[0], prepared_instruction)
    conv.append_message(conv.roles[1], None)
    prompt = [conv.get_prompt()]
    answers = model.generate(prepared_images, prompt, max_new_tokens=args.max_new_tokens,
                               do_sample=True, lengths=[length], num_beams=1)
    return answers[0]

def setup_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    cudnn.benchmark = False
    cudnn.deterministic = True

with open('test_configs/llama2_test_config.yaml') as file:
    config = yaml.load(file, Loader=yaml.FullLoader)
seed = config['run']['seed']
print("Seed:", seed)

# Initialize the model and visual processor.
model, vis_processor = init_model(args)
conv = CONV_VISION.copy()
conv.system = ""
inference_subtitles_folder = "inference_subtitles"
os.makedirs(inference_subtitles_folder, exist_ok=True)
existed_subtitles = {}
for sub in os.listdir(inference_subtitles_folder):
    existed_subtitles[sub.split('.')[0]] = True

In [ ]:
!pip install gradio
# === GRADIO INTERFACE: Define UI and connect backend ===

import gradio as gr

def process_video(video, question):
    if video is None:
        return "Please upload a video file."
    # Gradio may provide the video as a filepath or a file object.
    video_path = video if isinstance(video, str) else video.name
    result = run(video_path, question, model, vis_processor, gen_subtitles=args.add_subtitles)
    return result

iface = gr.Interface(
    fn=process_video,
    inputs=[
        gr.Video(label="Upload Video"),
        gr.Textbox(label="Enter your question", placeholder="Question")
    ],
    outputs="text",
    title="“Video Understanding and Language-based Contextual Answer Network",
    description="Upload a video and ask any question related to it. The model processes the video and generates an answer."
)

iface.launch()